# Plotting with bokeh

In [ ]:
# General imports

import numpy as np
from netCDF4 import Dataset
from postladim import ParticleFile

In [ ]:
# Bokeh imports

import bokeh
from bokeh.models.mappers import LogColorMapper
from bokeh.plotting import figure, show

# Notebook display
bokeh.io.output_notebook()

In [ ]:
# Files
particle_file = "out.nc"
grid_file = "../data/ocean_avg_0014.nc"

# Time step
t = 48  # 6 days (3 hours between output)

In [ ]:
# Read the particle_file

# Bokeh does not handle xarray DataArray, take values to make numpy array.
with ParticleFile(particle_file) as pf:
    X = pf.X[t].values
    Y = pf.Y[t].values

In [ ]:
p = figure(height=400, width=600, background_fill_color="skyblue", match_aspect=True)
p.circle(X, Y, color="red")
show(p)

In [ ]:
# Read bottom topography and land mask
with Dataset(grid_file) as nc:
    H = nc.variables["h"][:, :]
    mask = nc.variables["mask_rho"][:, :]
jmax, imax = mask.shape

In [ ]:
scale = 3

p = figure(match_aspect=True)
p.image(
    image=[mask],
    x=-0.5,
    y=-0.5,
    dw=imax,
    dh=jmax,
    palette=["Olive", "Skyblue"],
    # level="image",
)
p.circle(X, Y, color="red")

show(p)

In [ ]:
# Zoom to a subgrid with correct aspect ratio and user coordinates
# Turns off box zoom as it may destroy the aspect ratio

i0, i1, j0, j1 = 30, 160, 60, 120
scale = 4

p = figure(
    x_range=(i0 - 0.5, i1 - 0.5),
    y_range=(j0 - 0.5, j1 - 0.5),
    frame_width=scale * (i1 - i0),
    frame_height=scale * (j1 - j0),
    tools="pan, wheel_zoom, reset, save",
)

p.image(
    image=[mask],
    x=-0.5,
    y=-0.5,
    dw=imax,
    dh=jmax,
    palette=["Olive", "Skyblue"],
    # level="image",
)
p.circle(X, Y, color="red")

show(p)

In [ ]:
scale = 3
p = figure(
    x_range=(-0.5, imax - 0.5),
    y_range=(-0.5, jmax - 0.5),
    frame_width=scale * imax,
    frame_height=scale * jmax,
)

# Set land values of bathymetry to nan
H = np.where(mask > 0, H, np.nan)

# Define a blue color map with darker values at depth, Blues256 reversed.
# The map is logarithmic to show details in the shallow North Sea
cmap = LogColorMapper(palette=bokeh.palettes.Blues256[::-1])
# Set land color
cmap.nan_color = "Olive"

p.image(
    image=[H],
    x=-0.5,
    y=-0.5,
    dw=imax,
    dh=jmax,
    color_mapper=cmap,
)

# Display the particle positions
p.circle(X, Y, color="red")

show(p)